# Taller 1: Agentes que Planean y Búsquedas
### [Introducción a los Sistemas Inteligentes 2019-1](https://fagonzalezo.github.io/iis-2019-1/)
### Universidad Nacional de Colombia, Bogotá

---
* Rubén Camilo Buelvas Villa
* Jonathan Andrés Campo Rangel

---

In [1]:
!rm -r snake-ai-reinforcement
!git clone https://github.com/YuriyGuts/snake-ai-reinforcement.git
!mv snake-ai-reinforcement/snakeai .
!ls


Cloning into 'snake-ai-reinforcement'...
remote: Enumerating objects: 197, done.
remote: Total 197 (delta 0), reused 0 (delta 0), pack-reused 197
Receiving objects: 100% (197/197), 42.98 KiB | 4.30 MiB/s, done.
Resolving deltas: 100% (97/97), done.
mv: cannot move 'snake-ai-reinforcement/snakeai' to './snakeai': Directory not empty
sample_data  snakeai  snake-ai-reinforcement


El objetivo de este taller es construir un agente que planea que sea capaz de jugar el juego de *Snake*:

<img src="https://cloud.githubusercontent.com/assets/2750531/24808769/cc825424-1bc5-11e7-816f-7320f7bda2cf.gif" alt="Snake snapshot" width="320"/>

El agente tendrá acceso a todo el estado del ambiente. Debe usar esta información para hacer un plan y después actuar de acuerdo a este. 

Para esto vamos a usar como base este [proyecto](https://github.com/YuriyGuts/snake-ai-reinforcement) desarrollado por [Yuriy Guts](https://github.com/YuriyGuts).

En este caso vamos a utilizar la clase `Environment` tal como está definida en el paquete `snakeai.gameplay.environment`. El método `get_observation()` retorna una arreglo de `numpy` con la información de todo el campo de juego. 

## 1. Problema de búsqueda
Se debe modelar el problema de encontrar un plan (secuencia de acciones) para la serpiente como un problema de búsqueda. En particular el problema de búsqueda debe recibir la configuración actual del tablero la cual corresponde al estado inicial, las acciones corresponden a las acciones que puede ejecutar la serpiente, los estados finales son aquellos en que la serpiente se come la fruta.

El estado se debe representar como una tupla con el estado del tablero y el de la serpiente (una instancia de la clase `Snake` del paquete `snakeai.gameplay.entities`)

In [0]:
import matplotlib.pyplot as plt
import random
import heapq
import math
import sys
import copy
from collections import defaultdict, deque, Counter
from itertools import combinations

In [0]:
import numpy as np
from snakeai.gameplay.entities import SnakeAction, Snake, Point, SnakeDirection
from snakeai.gameplay.environment import Environment

class SnakeProblem(object):
    """The abstract class for a formal problem. A new domain subclasses this,
    overriding `actions` and `results`, and perhaps other methods.
    The default heuristic is 0 and the default action cost is 1 for all states.
    When yiou create an instance of a subclass, specify `initial`, and `goal` states 
    (or give an `is_goal` method) and perhaps other keyword args for the subclass."""

    def __init__(self, initial=None, **kwds): 
        self.__dict__.update(initial=initial, **kwds) 
        
    def actions(self, state):
        st=list(state)
        direction = st[1].direction
        [y, x] = list(st[1].head) 
        #borderX=len(st[0])
        #borderY=len(st[0][0])

        if direction == SnakeDirection.NORTH:
            return(["SnakeAction.MAINTAIN_DIRECTION"    for i in range(1) if st[0][x-1, y] != 3 and st[0][x-1, y] != 4] +
                   ["SnakeAction.TURN_LEFT"             for i in range(1) if st[0][x, y-1] != 3 and st[0][x, y-1] != 4] +
                   ["SnakeAction.TURN_RIGHT"            for i in range(1) if st[0][x, y+1] != 3 and st[0][x, y+1] != 4])
        
        elif direction == SnakeDirection.EAST:
            return(["SnakeAction.MAINTAIN_DIRECTION"    for i in range(1) if st[0][x, y+1] != 3 and st[0][x, y+1] != 4] +
                   ["SnakeAction.TURN_LEFT"             for i in range(1) if st[0][x-1, y] != 3 and st[0][x-1, y] != 4] +
                   ["SnakeAction.TURN_RIGHT"            for i in range(1) if st[0][x+1, y] != 3 and st[0][x+1, y] != 4])
        
        elif direction == SnakeDirection.SOUTH:
            return(["SnakeAction.MAINTAIN_DIRECTION"    for i in range(1) if st[0][x+1, y] != 3 and st[0][x+1, y] != 4] +
                   ["SnakeAction.TURN_LEFT"             for i in range(1) if st[0][x, y+1] != 3 and st[0][x, y+1] != 4] +
                   ["SnakeAction.TURN_RIGHT"            for i in range(1) if st[0][x, y-1] != 3 and st[0][x, y-1] != 4])
        
        elif direction == SnakeDirection.WEST:
            return(["SnakeAction.MAINTAIN_DIRECTION"    for i in range(1) if st[0][x, y-1] != 3 and st[0][x, y-1] != 4] +
                   ["SnakeAction.TURN_LEFT"             for i in range(1) if st[0][x+1, y] != 3 and st[0][x+1, y] != 4] +
                   ["SnakeAction.TURN_RIGHT"            for i in range(1) if st[0][x-1, y] != 3 and st[0][x-1, y] != 4])
        
        
        
    def result(self, state, action): 
        """The state that results from executing this action in this state."""
        
        result = copy.deepcopy(list(state))
        st = copy.deepcopy(list(state))
        [y, x] = list(st[1].head)
        [yt, xt] = list(st[1].tail)
        act = action
        direction = st[1].direction
        if direction == SnakeDirection.NORTH:
            if act == "SnakeAction.MAINTAIN_DIRECTION":                
                result[0][x-1, y] = 2
                result[0][x, y] = 3
                result[0][xt, yt] = 0
                result[1].move()              
            elif act == "SnakeAction.TURN_LEFT":
                result[0][x, y-1] = 2
                result[0][x, y] = 3
                result[0][xt, yt] = 0
                result[1].turn_left()
                result[1].move()
            elif act == "SnakeAction.TURN_RIGHT":
                result[0][x, y+1] = 2
                result[0][x, y] = 3
                result[0][xt, yt] = 0
                result[1].turn_right()
                result[1].move()
                
        elif direction == SnakeDirection.EAST:
            if act == "SnakeAction.MAINTAIN_DIRECTION":
                result[0][x, y+1] = 2
                result[0][x, y] = 3
                result[0][xt, yt] = 0
                result[1].move()              
            elif act == "SnakeAction.TURN_LEFT":
                result[0][x-1, y] = 2
                result[0][x, y] = 3
                result[0][xt, yt] = 0
                result[1].turn_left()
                result[1].move()
            elif act == "SnakeAction.TURN_RIGHT":
                result[0][x+1, y] = 2
                result[0][x, y] = 3
                result[0][xt, yt] = 0
                result[1].turn_right()
                result[1].move()
                
        elif direction == SnakeDirection.SOUTH:
            if act == "SnakeAction.MAINTAIN_DIRECTION":
                result[0][x+1, y] = 2
                result[0][x, y] = 3
                result[0][xt, yt] = 0
                result[1].move()              
            elif act == "SnakeAction.TURN_LEFT":
                result[0][x, y+1] = 2
                result[0][x, y] = 3
                result[0][xt, yt] = 0
                result[1].turn_left()
                result[1].move()
            elif act == "SnakeAction.TURN_RIGHT":
                result[0][x, y-1] = 2
                result[0][x, y] = 3
                result[0][xt, yt] = 0
                result[1].turn_right()
                result[1].move()
        else:
            if act == "SnakeAction.MAINTAIN_DIRECTION":
                result[0][x, y-1] = 2
                result[0][x, y] = 3
                result[0][xt, yt] = 0
                result[1].move()              
            elif act == "SnakeAction.TURN_LEFT":
                result[0][x+1, y] = 2
                result[0][x, y] = 3
                result[0][xt, yt] = 0
                result[1].turn_left()
                result[1].move()
            else:
                result[0][x-1, y] = 2
                result[0][x, y] = 3
                result[0][xt, yt] = 0
                result[1].turn_right()
                result[1].move()
        #if self.is_goal(state):
         # result[0][xt, yt] = 3
         # result[1].grow
        
        return result

    def is_goal(self, state):
        m = state[0]
        flag = True
        for i in m:
            if 1 in m:
                flag = False
        return flag
    
    def action_cost(self, s, a, s1): 
        return 1
    
    def h(self, node):               
        return 0
    
    def __str__(self):
        return '{}({!r}, {!r})'.format(
            type(self).__name__, self.initial, self.goal)

In [0]:
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.path_cost < other.path_cost
    
    
failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff',  path_cost=math.inf) # Indicates iterative deepening search was cut off.
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.action_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []  
    return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node in (cutoff, failure, None): 
        return []
    return path_states(node.parent) + [node.state]

In [5]:

inicial = {
  "field": [
    "##########",
    "#...#....#",
    "#.O#####.#",
    "#.###.#..#",
    "#...#S...#",
    "#....s.#.#",
    "#..##s.#.#",
    "#...#s.#.#",
    "#..##s...#",
    "##########"
  ],

  "initial_snake_length": 3,
  "max_step_limit": 1000,

  "rewards": {
    "timestep": 0,
    "ate_fruit": 1,
    "died": -1
  }
}
env = Environment(config=inicial, verbose=0)
env.new_episode()
env.get_observation()

array([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
       [4, 0, 1, 0, 4, 0, 0, 0, 0, 4],
       [4, 0, 1, 4, 4, 4, 4, 4, 0, 4],
       [4, 0, 4, 4, 4, 0, 4, 0, 0, 4],
       [4, 0, 0, 0, 4, 2, 0, 0, 0, 4],
       [4, 0, 0, 0, 0, 3, 0, 4, 0, 4],
       [4, 0, 0, 4, 4, 3, 0, 4, 0, 4],
       [4, 0, 0, 0, 4, 3, 0, 4, 0, 4],
       [4, 0, 0, 4, 4, 3, 0, 0, 0, 4],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]])

## 2. Algoritmos de búsqueda no informada
Escriba diferentes funciones para resolver el problema de búsqueda usando BFS e IDS. Pruebe el algoritmo con diferentes configuraciones del problema (diferentes tamaños del tablero, obstáculos, longitudes de la serpiente, etc.). Evalue y discuta los resultados. 

In [0]:
FIFOQueue = deque
def our_breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    node = Node(problem.initial)
    if problem.is_goal(problem.initial):
        return node
    frontier = FIFOQueue([node])
    tuples=[]
    for i in problem.initial[0]:
        tuples.append(tuple(i))
    doubletuple=(tuple(tuples), problem.initial[1])
    reached = {doubletuple}
    while frontier:
        node = frontier.pop()
        for child in expand(problem, node):
            s = child.state
            if problem.is_goal(s):
                return child
            tuples = []
            for i in s[0]:
                tuples.append(tuple(i))
            doubletuple = (tuple(tuples), s[1])
            s = {doubletuple}
            s = tuple(s)
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

In [0]:
LIFOQueue = list
def is_cycle(node, k=60):
    "Does this node form a cycle of length k or less?"
    def find_cycle(ancestor, k):

        return (ancestor is not None and k > 0 and
                (np.all(ancestor.state[0] == node.state[0]) or find_cycle(ancestor.parent, k - 1)))
    
    return find_cycle(node.parent, k)

def depth_limited_search(problem, limit=10):
    "Search deepest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    result = failure
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        elif len(node) >= limit:
            result = cutoff
        elif not is_cycle(node):
            for child in expand(problem, node):
                frontier.append(child)
    return result


def our_iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result

In [0]:
def solveProblemBFS(problem):
    """
    Recibe una instancia de SnakeProblem y retorna una lista con la secuencia de acciones que resuelve el problema.
    La solución debe ser óptima (mínimo número de pasos).
    """
    final_node = our_breadth_first_search(problem)
    estados = path_states(final_node)
    #print("Tablero inicial:\n ", estados[0][0])
    #print("Tablero final:\n ", estados[-1][0])
    pasos = path_actions(final_node)
    #print("La secuencia de acciones para la solución es:\n ")
    solucion = pasos
    return solucion

def solveProblemIDS(problem):
    """
    Recibe una instancia de SnakeProblem y retorna una lista con la secuencia de acciones que resuelve el problema.
    La solución debe ser óptima (mínimo número de pasos).
    """
    final_node = our_iterative_deepening_search(problem)
    estados = path_states(final_node)
    #print("Tablero inicial:\n ", estados[0][0])
    #print("Tablero final:\n ", estados[-1][0])
    pasos = path_actions(final_node)
    #print("La secuencia de acciones para la solución es:\n ")
    solucion = pasos
    return solucion


##Algunos estados iniciales

In [0]:
initial = (
             np.array([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
                       [4, 0, 0, 0, 4, 0, 0, 0, 0, 4],
                       [4, 0, 1, 4, 4, 4, 4, 4, 0, 4],
                       [4, 0, 4, 4, 4, 0, 4, 0, 0, 4],
                       [4, 0, 0, 0, 4, 2, 0, 0, 0, 4],
                       [4, 0, 0, 0, 0, 3, 0, 4, 0, 4],
                       [4, 0, 0, 4, 4, 3, 0, 4, 0, 4],
                       [4, 0, 0, 0, 4, 3, 0, 4, 0, 4],
                       [4, 0, 0, 4, 4, 3, 0, 0, 0, 4],
                       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]),
             Snake(Point(5, 4), length=5)
          )

initial2 = (
             np.array([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
                       [4, 0, 0, 0, 0, 0, 0, 0, 2, 4],
                       [4, 0, 4, 4, 4, 4, 4, 4, 3, 4],
                       [4, 0, 0, 0, 0, 0, 0, 4, 3, 4],
                       [4, 0, 4, 0, 4, 4, 0, 4, 3, 4],
                       [4, 0, 4, 0, 0, 0, 0, 4, 3, 4],
                       [4, 0, 4, 0, 1, 0, 0, 4, 3, 4],
                       [4, 0, 4, 4, 4, 4, 4, 4, 3, 4],
                       [4, 0, 0, 0, 0, 0, 0, 0, 3, 4],
                       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]),
             Snake(Point(8, 1), length=8)
          )

initial3 = (
             np.array([[4, 4, 4, 4, 4, 4], 
                       [4, 0, 0, 0, 0, 4], 
                       [4, 0, 4, 4, 0, 4], 
                       [4, 0, 4, 0, 0, 4], 
                       [4, 0, 4, 0, 4, 4], 
                       [4, 0, 4, 0, 0, 4], 
                       [4, 0, 4, 0, 0, 4], 
                       [4, 1, 4, 0, 4, 4], 
                       [4, 4, 4, 0, 0, 4], 
                       [4, 0, 0, 0, 4, 4],
                       [4, 0, 4, 4, 4, 4], 
                       [4, 0, 0, 0, 0, 4], 
                       [4, 0, 4, 4, 0, 4], 
                       [4, 4, 4, 4, 0, 4], 
                       [4, 2, 0, 4, 0, 4], 
                       [4, 3, 0, 4, 0, 4], 
                       [4, 3, 0, 4, 0, 4], 
                       [4, 3, 0, 4, 0, 4], 
                       [4, 3, 0, 0, 0, 4], 
                       [4, 4, 4, 4, 4, 4]]),
             Snake(Point(1, 14), length=5)
          )

initial4 = (
             np.array([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
                       [4, 2, 0, 0, 4, 0, 0, 0, 0, 4, 4, 4, 4, 4, 4],
                       [4, 3, 4, 0, 0, 4, 4, 4, 0, 0, 0, 0, 0, 4, 4],
                       [4, 3, 0, 4, 0, 0, 0, 4, 0, 4, 4, 4, 0, 4, 4],
                       [4, 3, 4, 0, 4, 4, 0, 4, 0, 4, 4, 4, 0, 4, 4],
                       [4, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
                       [4, 3, 4, 0, 0, 0, 0, 4, 0, 4, 4, 0, 4, 0, 4],
                       [4, 0, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 0, 4],
                       [4, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
                       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]),
             Snake(Point(1, 1), length=5)
           )

my_problem = SnakeProblem(initial=initial)
my_problem2 = SnakeProblem(initial=initial2)
my_problem3 = SnakeProblem(initial=initial3)
my_problem4 = SnakeProblem(initial=initial4)

##NOTA:
El problem3, en todos los casos (usando cualquier algoritmo) tarda un tiempo consiferablemente mayor en su solución, debido al tamaño del tablero.

##Solución usando BFS

In [10]:
solveProblemBFS(my_problem)

['SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT']

In [11]:
solveProblemBFS(my_problem2)

['SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT']

In [12]:
solveProblemBFS(my_problem3)

['SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAc

In [13]:
solveProblemBFS(my_problem4)

['SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DI

##Solución usando IDS

In [14]:
solveProblemIDS(my_problem)

['SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT']

In [15]:
solveProblemIDS(my_problem2)

['SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.TURN_RIGHT']

In [16]:
solveProblemIDS(my_problem3)

['SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAc

In [17]:
solveProblemIDS(my_problem4)

['SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DI

##Discusión

Los dos algoritmos de búsqueda envían la misma solución (la óptima), sin embarga el IDS es mucho más lento que el BFS.


## 3. Algoritmos de búsqueda informada
Escriba una función para resolver el problema de búsqueda usando A*. Prueba al menos dos funciones heurísticas diferentes. Las heurísticas deben ser admisibles. Pruebe el algoritmo con diferentes configuraciones del problema (diferentes tamaños del tablero, obstáculos, longitudes de la serpiente, etc.). Evalue y discuta los resultados. 

In [0]:
class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = [] # a heap of (score, item) pairs
        for item in items:
            self.add(item)
    
    def add(self, item):
        """Add item to the queuez."""
        pair = (self.key(item), item)
        heapq.heappush(self.items, pair)

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]
    
    def top(self): return self.items[0][1]

    def __len__(self): return len(self.items)

def our_best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    node = Node(problem.initial)
    frontier = PriorityQueue([node], key=f)
    tuples=[]
    for i in problem.initial[0]:
        tuples.append(tuple(i))
    doubletuple = (tuple(tuples), problem.initial[1])
    reached = {doubletuple: node}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            tuples = []
            for i in s[0]:
                tuples.append(tuple(i))
            doubletuple = (tuple(tuples), s[1])
            s = {doubletuple}
            s = tuple(s)
           # child.state = s
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure

def g(n): return n.path_cost
  
def astar_search(problem, h=None):
    """Search nodes with minimum f(n) = g(n) + h(n)."""
    #h = h or problem.h
    h=h
    g=1
    return our_best_first_search(problem, f=lambda n: g + h)


def manhattan(a, b):
    xdis = abs(list(a)[0]-list(b)[0])
    ydis = abs(list(a)[1]-list(b)[1])
    return xdis + ydis

def real_dist(a, b):
    x1 = list(a)[0]
    x2 = list(b)[0]
    y1 = list(a)[1]
    y2 = list(b)[1]
    return math.sqrt((x1-x2)**2+(y1-y2)**2)


def solveProblemAStar_h1(problem):
    """
    Recibe una instancia de SnakeProblem y retorna una lista con la secuencia de acciones que resuelve el problema.
    La solución debe ser óptima (mínimo número de pasos).      
    """
    head = problem.initial[1].head
    for i in range(len(problem.initial[0])):
        for j in range(len(problem.initial[0][i])):
            if problem.initial[0][i, j] == 1:
                fruit = Point(i, j)

    final_node = astar_search(problem, manhattan(head, fruit))
    estados = path_states(final_node)
    #print("Tablero inicial:\n ", estados[0][0])
    #print("Tablero final:\n ", estados[-1][0])
    pasos = path_actions(final_node)
    #print("La secuencia de acciones para la solución es:\n ")
    solucion = pasos
    return solucion    

def solveProblemAStar_h2(problem):
    """
    Recibe una instancia de SnakeProblem y retorna una lista con la secuencia de acciones que resuelve el problema.
    La solución debe ser óptima (mínimo número de pasos).
    """
    head = problem.initial[1].head
    for i in range(len(problem.initial[0])):
        for j in range(len(problem.initial[0][i])):
            if problem.initial[0][i, j] == 1:
                fruit = Point(i, j)

    final_node = astar_search(problem, real_dist(head, fruit))
    estados = path_states(final_node)
    #print("Tablero inicial:\n ", estados[0][0])
    #print("Tablero final:\n ", estados[-1][0])
    pasos = path_actions(final_node)
    #print("La secuencia de acciones para la solución es:\n ")
    solucion = pasos
    return solucion    

##Heuristica 1(Manhattan)

In [19]:
solveProblemAStar_h1(my_problem)

['SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT']

In [20]:
solveProblemAStar_h1(my_problem2)

['SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.TURN_RIGHT']

In [21]:
solveProblemAStar_h1(my_problem3)

['SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAc

In [22]:
print(solveProblemAStar_h1(my_problem4))

['SnakeAction.TURN_RIGHT', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.TURN_RIGHT', 'SnakeAction.TURN_LEFT', 'SnakeAction.TURN_RIGHT', 'SnakeAction.TURN_LEFT', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.TURN_RIGHT', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.TURN_LEFT', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.TURN_RIGHT', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.TURN_RIGHT', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION', 'SnakeAction.MAINTAIN_DIRECTION']


##Heuristica 2(Distancia Real)

In [23]:
solveProblemAStar_h2(my_problem)

['SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT']

In [24]:
solveProblemAStar_h2(my_problem2)

['SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.TURN_RIGHT']

In [25]:
solveProblemAStar_h2(my_problem3)

['SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAc

In [26]:
solveProblemAStar_h2(my_problem4)

['SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_LEFT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.TURN_RIGHT',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DIRECTION',
 'SnakeAction.MAINTAIN_DI

##Discusión
A pesar de que Manhattan sea mucho menos admisible que la distancia euclidiana, el algoritmo A* da el mismo resultado utilizando cualquiera de las dos heurísticas, en tiempos similares y para diferentes escenarios.

## 4. Agente que planea para jugar Snake 
Desarrolle un agente para jugar Snake que construya un plan cada vez que aparece una nueva fruta. El agente debe precalcular el plan y después ejecutarlo. Cuando termine de ejecutar las acciones del plan, debe volver a calcular un nuevo plan. 

In [0]:
from snakeai.agent import AgentBase

class MasterMindPlanningAgent(AgentBase):
    """ Represents a Snake agent that takes a random action at every step. """

    def __init__(self):
        self.step = 0
        self.c = 0
        self.actions = []
        pass

    def begin_episode(self):
        pass

    def act(self, observation, reward):
        
        
        if len(self.actions) == self.c:
            self.actions = self.plan(observation)
            self.c = 0
            if len(self.actions) == 0:
                env.is_episode_end = True

        if self.actions[self.c] == "SnakeAction.TURN_RIGHT":
            action = SnakeAction.TURN_RIGHT
        elif self.actions[self.c] == "SnakeAction.TURN_LEFT":
            action = SnakeAction.TURN_LEFT
        elif self.actions[self.c] == "SnakeAction.MAINTAIN_DIRECTION":
            action = SnakeAction.MAINTAIN_DIRECTION
        self.step += 1
        self.c += 1
        return action
    
    def plan(self, observation):
        my_problem = SnakeProblem(initial=observation)
        plan = solveProblemAStar_h2(my_problem)
        return plan

    def end_episode(self):
        pass

Utilice el siguiente código para ejecutar el agente. Evaluelo en diferentes tableros iniciales. Reporte las estadísticas y compare el desempeño con los agentes al azar y reactivos de la [Práctica 1](https://colab.research.google.com/drive/14bQITZS1wVLYJEKG4jNVsWbsBGryuOvY). Analice y discuta los resultados.

In [28]:
from snakeai.gameplay.entities import ALL_SNAKE_ACTIONS, Point
import numpy as np
import random

class EnvironmentFull(Environment):
    """
    Full observation environment. Same as base class environment, overloads 
    `get_observation` so that it returns the Field array and the Snake.
    (From Environment doc): Represents the RL environment for the Snake game that implements the game logic,
    provides rewards for the agent and keeps track of game statistics.
    """
    def __init__(self, config, verbose=0):
        super().__init__(config, verbose)

    def get_observation(self):
        """ Observe the state of the environment. """
        return (np.copy(self.field._cells), self.snake)
    
    def show_field(self):
        return self.field.__str__()
    
def play(env, agent, num_episodes=1, verbose=1):
    """
    Play a set of episodes using the specified Snake agent.
    Use the non-interactive command-line interface and print the summary statistics afterwards.
    
    Args:
        env: an instance of Snake environment.
        agent: an instance of Snake agent.
        num_episodes (int): the number of episodes to run.
    """

    fruit_stats = []

    print()
    print('Playing:')

    for episode in range(num_episodes):
        timestep = env.new_episode()
        agent.begin_episode()
        game_over = False
        step = 0
        while not game_over:
            try:
                if verbose > 0:
                    print("------ Step ", step, " ------")
                    print (env.show_field())
                    print ("Observation:", env.get_observation())
                    print ("Head:", env.snake.head)
                    print ("Direction:", env.snake.direction)
                step += 1
                action = agent.act(timestep.observation, timestep.reward)
                env.choose_action(action)
                timestep = env.timestep()
                game_over = timestep.is_episode_end
            except:
                game_over = True

        fruit_stats.append(env.stats.fruits_eaten)

        summary = '******* Episode {:3d} / {:3d} | Timesteps {:4d} | Fruits {:2d}'
        print(summary.format(episode + 1, num_episodes, env.stats.timesteps_survived, env.stats.fruits_eaten))

    print()
    print('Fruits eaten {:.1f} +/- stddev {:.1f}'.format(np.mean(fruit_stats), np.std(fruit_stats)))
    
inicial = {
  "field": [
    "#######",
    "#.....#",
    "#.....#",
    "#..S..#",
    "#.....#",
    "#.....#",
    "#######"
  ],

  "initial_snake_length": 2,
  "max_step_limit": 1000,

  "rewards": {
    "timestep": -0.01,
    "ate_fruit": 1,
    "died": -1
  }
}

env = EnvironmentFull(config=inicial, verbose=0)
agent = MasterMindPlanningAgent()
play(env, agent, num_episodes= 10, verbose=0)


Playing:
******* Episode   1 /  10 | Timesteps   32 | Fruits  9
******* Episode   2 /  10 | Timesteps   58 | Fruits 14
******* Episode   3 /  10 | Timesteps   31 | Fruits  9
******* Episode   4 /  10 | Timesteps   17 | Fruits  8
******* Episode   5 /  10 | Timesteps   22 | Fruits  8
******* Episode   6 /  10 | Timesteps   32 | Fruits  9
******* Episode   7 /  10 | Timesteps   41 | Fruits  8
******* Episode   8 /  10 | Timesteps   24 | Fruits  6
******* Episode   9 /  10 | Timesteps   55 | Fruits 13
******* Episode  10 /  10 | Timesteps   64 | Fruits 16

Fruits eaten 10.0 +/- stddev 3.0


In [29]:
inicial = {
  "field": [
    "#################",
    "#...............#",
    "#.###...#....#..#",
    "#..S...##....#..#",
    "#...............#",
    "#........###....#",
    "#################"
  ],

  "initial_snake_length": 2,
  "max_step_limit": 1000,

  "rewards": {
    "timestep": -0.01,
    "ate_fruit": 1,
    "died": -1
  }
}

env = EnvironmentFull(config=inicial, verbose=0)
agent = MasterMindPlanningAgent()
play(env, agent, num_episodes= 2, verbose=0)


Playing:
******* Episode   1 /   2 | Timesteps   94 | Fruits 16
******* Episode   2 /   2 | Timesteps   97 | Fruits 18

Fruits eaten 17.0 +/- stddev 1.0


In [30]:
inicial = {
  "field": [
    "#############",
    "#...........#",
    "#.##...#....#",
    "#..S.###....#",
    "##..........#",
    "#.......###.#", 
    "#..#........#",
    "#..#....###.#",
    "#...........#",
    "#############"
  ],

  "initial_snake_length": 2,
  "max_step_limit": 1000,

  "rewards": {
    "timestep": -0.01,
    "ate_fruit": 1,
    "died": -1
  }
}

env = EnvironmentFull(config=inicial, verbose=0)
agent = MasterMindPlanningAgent()
play(env, agent, num_episodes= 1, verbose=0)


Playing:
******* Episode   1 /   1 | Timesteps  256 | Fruits 32

Fruits eaten 32.0 +/- stddev 0.0


In [31]:
inicial = {
  "field": [
    "###########",
    "#.........#",
    "#.........#",
    "#.S.......#",
    "#.........#",
    "#.........#", 
    "#.........#",
    "#.........#",
    "#.........#",
    "###########"
  ],

  "initial_snake_length": 2,
  "max_step_limit": 1000,

  "rewards": {
    "timestep": -0.01,
    "ate_fruit": 1,
    "died": -1
  }
}

env = EnvironmentFull(config=inicial, verbose=0)
agent = MasterMindPlanningAgent()
play(env, agent, num_episodes= 1, verbose=1)


Playing:
------ Step  0  ------
###########
#.........#
#.........#
#.S.......#
#.s.O.....#
#.........#
#.........#
#.........#
#.........#
###########
Observation: (array([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
       [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
       [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
       [4, 0, 2, 0, 0, 0, 0, 0, 0, 0, 4],
       [4, 0, 3, 0, 1, 0, 0, 0, 0, 0, 4],
       [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
       [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
       [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
       [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
       [4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]]), <snakeai.gameplay.entities.Snake object at 0x7fc0e29ca7b8>)
Head: Point(x=2, y=3)
Direction: Point(x=0, y=-1)
------ Step  1  ------
###########
#.........#
#.........#
#.sS......#
#...O.....#
#.........#
#.........#
#.........#
#.........#
###########
Observation: (array([[4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
       [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
       [4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4],
       [4, 0, 3, 2

En este escenario, aun sin obstáculos, por el tamaño del tablero, el agente llegaba a estados en los que se tardaba mucho tiempo en encontrar la solución.

##Discusión
El árbol de búsqueda se expande demasiado rápido a medida que el tamaño del tablero aumenta. Como el tablero es más grande, la serpiente puede realizar muchos más pasos sin morir, esto hace que el tiempo de ejecución sea muy largo. A partir del paso 200 aproximadamente, Colab empieza a sufrir por falta de memoria.

Además, en una configuracion en  la que la serpiente tiene un tamaño considerablemente grande y la fruta aparece entre los obstáculos y su cuerpo, usualmente detrás de ella, el agente tarda demasiado en encontrar un plan para coger la fruta.


In [0]:
#------ Step  204  ------
#############
#...........#
#.##...#....#
#....###....#
##..........#
#.....s.###.#
#ssssss.###.#
#s.#.ss.###.#
#s.#.sssS...#
#ss#.sss###.#
#.s#.s..###.#
#Ossss......#
#############

Para esta configurción tardó un poco menos de 3 minutos pensando el siguiente paso, y para llegar a ella ya habían pasado cerca de 10 minutos.

Como conclusión: El algoritmo A*, con una buena heurística, es el mejor (más rápido y óptimo), sin embargo, para problemas "complejos" el tiempo que tarda en encontrar la solución es muy alto.